In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()] )

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [6]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and easy-to-make dish - Chicken Tikka Masala.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) tomato sauce
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1/2 tsp turmeric
- 1/2 tsp paprika
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (for serving)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, turmeric, paprika, cayenne pepper, salt, and pepper. Add the chicken pieces and mix well to coat. Marinate for at least 30 minutes, or 

AIMessageChunk(content="To make this Chicken Tikka Masala recipe vegetarian, we can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (use plant-based yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) tomato sauce\n- 1 tbsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1/2 tsp turmeric\n- 1/2 tsp paprika\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Instead of marinating chicken, marinate the tofu or paneer in the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriande